In [3]:
import sys
# This forces pip to install in the currently running Python environment
!{sys.executable} -m pip install yfinance --upgrade


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import warnings

# Suppress annoying warnings
warnings.filterwarnings("ignore")

# --- 1. MODEL DEFINITION ---
class HybridModel(nn.Module):
    def __init__(self, num_features, lstm_hidden=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=lstm_hidden, batch_first=True)
        self.sent_fc = nn.Sequential(
            nn.Linear(num_features, 32), nn.ReLU(), nn.Dropout(0.2)
        )
        self.final = nn.Sequential(
            nn.Linear(lstm_hidden + 32, 64), nn.ReLU(), nn.Linear(64, 1)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        sent = self.sent_fc(x[:, -1, :])
        cat = torch.cat([lstm_out[:, -1, :], sent], dim=1)
        return self.final(cat).squeeze(1)

# --- 2. SETUP ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HybridModel(num_features=12, lstm_hidden=64).to(device)
try:
    model.load_state_dict(torch.load('best_hybrid_model.pth'))
    model.eval()
except:
    print("❌ Error: 'best_hybrid_model.pth' not found.")
    # Exit logically or handle error
    
# --- 3. PREDICTION ENGINE ---
def get_signal(ticker):
    try:
        df = yf.download(ticker, period="1y", interval="1d", progress=False)
        if len(df) < 60: return None

        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.get_level_values(0)
        df = df.rename(columns={'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'})
        
        # Technicals
        df['return'] = df['close'].pct_change()
        df['ma5'] = df['close'].rolling(window=5).mean()
        df['ma10'] = df['close'].rolling(window=10).mean()
        df['volatility'] = df['close'].rolling(window=5).std()
        
        # Neutral Sentiment
        df['sent_mean'] = 0.5; df['sent_sum'] = 0.5; df['sent_count'] = 0.5
        df.dropna(inplace=True)
        
        # Scaling
        cols = ['open','high','low','close','volume','sent_mean','sent_sum','sent_count','return','ma5','ma10','volatility']
        scaling_df = df.copy()
        for c in cols:
            _min, _max = scaling_df[c].min(), scaling_df[c].max()
            scaling_df[c] = (scaling_df[c] - _min) / (_max - _min) if _max > _min else 0.0

        # Auto-Calibration (Last 5 days baseline)
        recent_preds = []
        with torch.no_grad():
            for i in range(5, 0, -1):
                win = scaling_df.iloc[-(30+i):-i][cols]
                if len(win)==30:
                    t = torch.tensor(win.values, dtype=torch.float32).unsqueeze(0).to(device)
                    recent_preds.append(model(t).item())
            
            # Today's Prediction
            last_30 = scaling_df.iloc[-30:][cols]
            t_now = torch.tensor(last_30.values, dtype=torch.float32).unsqueeze(0).to(device)
            pred_now = model(t_now).item()

        baseline = np.mean(recent_preds)
        signal = "🟢 BUY " if pred_now > baseline else "🔴 SELL"
        strength = abs(pred_now - baseline) / baseline * 100
        return signal, strength, pred_now
    except Exception as e:
        return None

# --- 4. RUN REPORT ---
watchlist = ['NVDA', 'TSLA', 'AAPL', 'AMZN', 'GOOGL', 'BTC-USD', 'ETH-USD']

print(f"{'TICKER':<10} | {'SIGNAL':<8} | {'STRENGTH':<10}")
print("-" * 35)

for ticker in watchlist:
    res = get_signal(ticker)
    if res:
        signal, strength, _ = res
        print(f"{ticker:<10} | {signal} | {strength:>6.2f}%")
    else:
        print(f"{ticker:<10} | ⚠️ DATA ERROR")

TICKER     | SIGNAL   | STRENGTH  
-----------------------------------
NVDA       | 🟢 BUY  |   5.82%
TSLA       | 🔴 SELL |   1.40%
AAPL       | 🔴 SELL |   7.15%
AMZN       | 🟢 BUY  |   9.94%
GOOGL      | 🔴 SELL |   2.14%
BTC-USD    | 🔴 SELL |   6.08%
ETH-USD    | 🔴 SELL |  12.43%
